<a href="https://colab.research.google.com/github/OtolaHan/Otola_Han_KAAN/blob/main/_homework_1_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание №1: линейная регрессия и векторное дифференцирование (10 баллов).

* Максимальное количество баллов за задания в ноутбуке - 11, но больше 10 оценка не ставится, поэтому для получения максимальной оценки можно сделать не все задания.

* Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [1]:
import numpy as np

## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [9]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

1.8660016528897028e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [11]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error

# Подбор гиперпараметров
alpha = 1e-5
max_iter = 1000
tol = 1e-3

reg_sgd = SGDRegressor(alpha=alpha, max_iter=max_iter, tol=tol, random_state=42)
reg_sgd.fit(X, y)
mse_sgd = mean_squared_error(y, reg_sgd.predict(X))

print(f"MSE for SGDRegressor: {mse_sgd}")

MSE for SGDRegressor: 3.2180558465576105e-06


***Задание 1 (0.5 балла).*** Объясните, чем вызвано различие двух полученных значений метрики?

***Задание 2 (0.5 балла).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс.

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и второй варианты) или l2 (третий и четвертый варианты) регуляризацию.

In [12]:
class LinearRegression(object):
    def __init__(self, alpha=0.0001, l_ratio=0.001, tol=0.001, max_iter=1000,
                 regularization='l2'):
        '''
        Для начала необходимо инициализировать параметры
        alpha - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''

        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter
        self.regularization = regularization
        self.weights = None

    def fit(self, X, y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''

        X = np.insert(X, 0, 1, axis=1)
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)

        for i in range(self.max_iter):
            predictions = X.dot(self.weights)
            errors = predictions - y
            gradient = X.T.dot(errors) / len(y)

            if self.regularization == 'l2':
                gradient += self.l_ratio * self.weights
            elif self.regularization == 'l1':
                gradient += self.l_ratio * np.sign(self.weights)

            new_weights = self.weights - self.alpha * gradient

            if np.linalg.norm(new_weights - self.weights) < self.tol or \
               np.abs(np.mean(errors ** 2) - np.mean((X.dot(new_weights) - y) ** 2)) < self.tol:
                break

            self.weights = new_weights

    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''

        X = np.hstack((np.ones((X.shape[0], 1)), X))
        return X.dot(self.weights)

In [14]:
my_reg = LinearRegression(regularization='l2')
my_reg.fit(X, y)
predictions = my_reg.predict(X)
print('You are amazing! Great work!')

You are amazing! Great work!


***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки

* с l1-регуляризацией (from sklearn.linear_model import Lasso, **первый и второй вариант**) или с l2-регуляризацией (from sklearn.linear_model import Ridge, **третий и четвертый вариант**)
* со значением параметра регуляризации **0.1 - для первого и третьего варианта, 0.01 - для второго и четвертого варианта**.

Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

ridge_reg = Ridge(alpha=0.01)
ridge_reg.fit(X_scaled, y)
ridge_pred = ridge_reg.predict(X_scaled)
ridge_mse = mean_squared_error(y, ridge_pred)

my_reg = LinearRegression(l_ratio=0.01)
my_reg.fit(X_scaled, y)
my_pred = my_reg.predict(X_scaled)
my_mse = mean_squared_error(y, my_pred)

print(f'Ridge MSE: {ridge_mse}')
print(f'My Linear Regression MSE: {my_mse}')
print("Results are similar" if np.isclose(ridge_mse, my_mse, atol=1e-3) else "Results are different")

Ridge MSE: 3.001299234616173e-08
My Linear Regression MSE: 23847.67361153917
Results are different


***Задание 6* (1 балл).***
Пусть $P, Q \in \mathbb{R}^{n\times n}$. Найдите $\nabla_Q tr(PQ)$

***Задание 7* (1 балл).***
Пусть $x, y \in \mathbb{R}^{n}, M \in \mathbb{R}^{n\times n}$. Найдите $\nabla_M x^T M y$

Решения заданий 6 и 7 можно написать на листочке и отправить в anytask вместе с заполненным ноутбуком.